In [34]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Any, Text, Dict, List
import plotly.express as px
import plotly.graph_objects as go

#### Illustrating the scale of electricity consumption according to subscribed power

In [35]:
df_theta = pd.read_excel("C:/Users/idris/Desktop/ENSAE/S1_3A/Cloud_Computing/EnergyBot/consumption_prediction/data/raw/Theta.xls")#, engine='openpyxl')

df_theta = df_theta[df_theta["Sous Profil"].isin(['RES1-P1', 'RES2-P1', 'RES11-P1', 'RES2-P2'])].reset_index(drop=True)

df_theta.columns = ["SOUS_PROFIL", "THETA"]

df_theta["SOUS_PROFIL"].replace(["RES1-P1", "RES11-P1", "RES2-P1", "RES2-P2"], ["RES1", "RES11", "RES2_HP", "RES2_HC"], inplace=True)

In [36]:
df_theta

,SOUS_PROFIL,THETA
0,RES11,0.05978
1,RES1,0.05210
2,RES2_HP,0.05755
3,RES2_HC,0.04351


In [37]:
df_ponderation_heure = pd.DataFrame({"SOUS_PROFIL":["RES1", "RES11", "RES2_HP", "RES2_HC"],
                                "PROP_HEURE": np.array([1, 1, 5840/8760, 2920/8760])})


In [38]:
df_ponderation_heure

,SOUS_PROFIL,PROP_HEURE
0,RES1,1.000000
1,RES11,1.000000
2,RES2_HP,0.666667
3,RES2_HC,0.333333


In [39]:
df_profile_coeff = pd.read_csv("C:/Users/idris/Desktop/ENSAE/S1_3A/Cloud_Computing/EnergyBot/consumption_prediction/data/raw/coefficients-des-profils.csv", sep=";")

In [40]:
df_profile_coeff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70080 entries, 0 to 70079
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HORODATE               70080 non-null  object 
 1   SOUS_PROFIL            70080 non-null  object 
 2   CATEGORIE              70080 non-null  object 
 3   COEFFICIENT_PREPARE    70080 non-null  float64
 4   COEFFICIENT_AJUSTE     70080 non-null  float64
 5   COEFFICIENT_DYNAMIQUE  70080 non-null  float64
dtypes: float64(3), object(3)
memory usage: 3.2+ MB


In [41]:
df_profile_coeff.head()

,HORODATE,SOUS_PROFIL,CATEGORIE,COEFFICIENT_PREPARE,COEFFICIENT_AJUSTE,COEFFICIENT_DYNAMIQUE
0,2021-09-23T22:00:00+02:00,RES11_BASE,Residentiel,0.9889,0.9889,0.776897
1,2021-09-24T09:00:00+02:00,RES11_BASE,Residentiel,0.9026,0.9026,0.853975
2,2021-09-24T13:00:00+02:00,RES11_BASE,Residentiel,0.9219,0.9219,0.832210
3,2021-09-24T23:30:00+02:00,RES11_BASE,Residentiel,0.7311,0.7311,0.606208
4,2021-09-25T01:30:00+02:00,RES11_BASE,Residentiel,0.5510,0.5510,0.480508


In [42]:
df_profile_coeff.drop(["CATEGORIE", "COEFFICIENT_PREPARE", "COEFFICIENT_AJUSTE"], axis=1, inplace=True)

In [43]:
df_profile_coeff["HORODATE"] = df_profile_coeff["HORODATE"].apply(lambda x: datetime.strptime(x[:10], "%Y-%m-%d"))

df_profile_coeff["SOUS_PROFIL"].replace(["RES1_BASE", "RES11_BASE"], ["RES1", "RES11"], inplace=True)

df_profile_coeff = df_profile_coeff.sort_values(["HORODATE", "SOUS_PROFIL"]).reset_index(drop=True)

df_profile_coeff = df_profile_coeff.merge(df_theta, how="left").merge(df_ponderation_heure, how="left")

In [44]:
df_profile_coeff

,HORODATE,SOUS_PROFIL,COEFFICIENT_DYNAMIQUE,THETA,PROP_HEURE
0,2021-01-01,RES1,0.796208,0.05210,1.000000
1,2021-01-01,RES1,1.365907,0.05210,1.000000
2,2021-01-01,RES1,1.748989,0.05210,1.000000
3,2021-01-01,RES1,1.059245,0.05210,1.000000
4,2021-01-01,RES1,0.934897,0.05210,1.000000
...,...,...,...,...,...
70075,2021-12-31,RES2_HP,0.534729,0.05755,0.666667
70076,2021-12-31,RES2_HP,0.022360,0.05755,0.666667
70077,2021-12-31,RES2_HP,1.859858,0.05755,0.666667
70078,2021-12-31,RES2_HP,1.273187,0.05755,0.666667


In [45]:
def consommation(ps: int) -> pd.DataFrame:
    df_conso = df_profile_coeff.groupby(["HORODATE", "SOUS_PROFIL"]).mean().reset_index()

    df_conso["CONSOMMATION_PAR_PS"] = df_conso["COEFFICIENT_DYNAMIQUE"] * df_conso["THETA"]
    df_conso["PS"] = ps
    df_conso["CONSOMMATION"] = 24 * df_conso["PROP_HEURE"] *  df_conso["CONSOMMATION_PAR_PS"] * df_conso["PS"]

    df_conso = df_conso[["HORODATE", "SOUS_PROFIL", "CONSOMMATION"]]
    df_conso["SOUS_PROFIL"].replace(["RES1", "RES11", "RES2_HP", "RES2_HC"], ["RES1", "RES11", "RES2", "RES2"], inplace=True)
    df_conso = df_conso.groupby(["HORODATE", "SOUS_PROFIL"]).sum().reset_index()
    df_conso["PS"] = ps

    return df_conso

In [46]:
# aggregat de la consommation
consommation(9)[["SOUS_PROFIL", "CONSOMMATION"]].groupby("SOUS_PROFIL").agg(CONSOMMATION_TOTALE_ANNUELLE=("CONSOMMATION", "sum"), CONSOMMATION_MOYENNE_JOURNALIERE=("CONSOMMATION", "mean"))

,CONSOMMATION_TOTALE_ANNUELLE,CONSOMMATION_MOYENNE_JOURNALIERE
SOUS_PROFIL,,
RES1,4045.175964,11.082674
RES11,5071.738859,13.895175
RES2,4181.458576,11.456051


In [47]:
fig = px.line(consommation(9), x='HORODATE', 
              y="CONSOMMATION", color="SOUS_PROFIL", width=950, height=500)
#fig.update_layout(title="Série journalière du coefficient dynamique entre 01 décembre 2021 et le 31 mars")
fig.show()